In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from torch.utils.data import TensorDataset, DataLoader # Import TensorDataset and DataLoader

In [ ]:
# Load the dataset
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)

# Make target variable
y = df.pop('target')

# Prepare features
list_numerical = ['age', 'thalach', 'trestbps', 'chol', 'oldpeak']
X = df[list_numerical]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler().fit(X_train[list_numerical])
X_train[list_numerical] = scaler.transform(X_train[list_numerical])
X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Using long for classification targets

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create a DataLoader for the training set
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Define the model
class HeartModel(nn.Module):
    def __init__(self):
        super(HeartModel, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(in_features=5, out_features=64)  # 5 features as input, 64 units in hidden layer
        self.fc2 = nn.Linear(in_features=64, out_features=32)  # 64 units in hidden layer, 32 units in second hidden layer
        self.fc3 = nn.Linear(in_features=32, out_features=1)   # 32 units to 1 output (binary classification)
        self.relu = nn.ReLU()  # ReLU activation function
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation function for binary output


    def forward(self, x):
        x = self.relu(self.fc1(x))  # Apply first linear layer + ReLU
        x = self.relu(self.fc2(x))  # Apply second linear layer + ReLU
        x = self.sigmoid(self.fc3(x))  # Apply final linear layer + Sigmoid for binary output

        return x

In [ ]:
model = HeartModel()

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

In [ ]:
# Train the model
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0  # Track epoch loss for reporting
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        # Squeeze the output to match the target shape
        outputs = outputs.squeeze(1)  # Remove the extra dimension

        # Convert y_batch to float
        y_batch = y_batch.type(torch.float32)

        loss = criterion(outputs, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate batch loss
        epoch_loss += loss.item()

    # Print epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

Epoch 1/15, Loss: 0.7071
Epoch 2/15, Loss: 0.6521
Epoch 3/15, Loss: 0.6114
Epoch 4/15, Loss: 0.5639
Epoch 5/15, Loss: 0.5300
Epoch 6/15, Loss: 0.4890
Epoch 7/15, Loss: 0.4643
Epoch 8/15, Loss: 0.4601
Epoch 9/15, Loss: 0.4358
Epoch 10/15, Loss: 0.4253
Epoch 11/15, Loss: 0.4082
Epoch 12/15, Loss: 0.4161
Epoch 13/15, Loss: 0.4229
Epoch 14/15, Loss: 0.4020
Epoch 15/15, Loss: 0.3991


In [ ]:
# Evaluate the model
def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        y_pred = model(X)
        # Get predicted class based on `y_pred` (`y_pred` are probablities)
        y_pred_classes = (y_pred >= 0.5).int() # Convert probabilities to class labels (0 or 1)
    return accuracy_score(y.numpy(), y_pred_classes.numpy())

In [ ]:
# Compute permutation importance
def permutation_importance(model, X, y):
    baseline_score = evaluate_model(model, X, y)
    importances = []

    for i in range(X.shape[1]):
        X_permuted = X.clone()
        # Replacethe i column of the data with shuffled data
        X_permuted[:, i] = ...
        # Evaluate the model on the shuffled data
        score = ...
        importances.append(baseline_score - score)

    return np.array(importances)

In [ ]:
from tqdm import tqdm

# Fonction pour calculer l'importance des caractéristiques via la permutation
def permutation_importance(model, X_test, y_test, metric=accuracy_score):
    baseline_preds = model(X_test).squeeze().detach().numpy()  # Obtenez les prédictions du modèle
    baseline_preds = (baseline_preds > 0.5).astype(int)  # Convertir les probabilités en prédictions binaires
    baseline_score = metric(y_test.numpy(), baseline_preds)  # Calculer la précision de base

    importances = []
    for i in range(X_test.shape[1]):  # Itérer sur chaque caractéristique
        X_test_permuted = X_test.clone()
        X_test_permuted[:, i] = X_test_permuted[:, i][torch.randperm(X_test_permuted.size(0))]  # Mélanger la caractéristique

        permuted_preds = model(X_test_permuted).squeeze().detach().numpy()  # Prédictions après permutation
        permuted_preds = (permuted_preds > 0.5).astype(int)  # Convertir les probabilités en prédictions binaires
        permuted_score = metric(y_test.numpy(), permuted_preds)  # Calculer la précision après permutation

        importance = baseline_score - permuted_score  # L'importance est la baisse de score
        importances.append(importance)

    return np.array(importances)

# Calcul de l'importance des caractéristiques
importances = [permutation_importance(model, X_test_tensor, y_test_tensor) for _ in tqdm(range(100))]  # Répéter 100 fois
importances = np.stack(importances).mean(axis=0)  # Moyenne des importances sur 100 itérations

# Afficher l'importance des caractéristiques
for i, importance in enumerate(importances):
    print(f"Feature {list_numerical[i]}: {importance:.4f}")



100%|██████████| 100/100 [00:00<00:00, 199.08it/s]


Feature age: -0.0093
Feature thalach: 0.0482
Feature trestbps: 0.0087
Feature chol: -0.0007
Feature oldpeak: 0.1049
